# DEAP_Valence

In [1]:
###libraries

#Data manipulation
import pandas as pd
import numpy as np
from collections import Counter
from sklearn import preprocessing
import sys
import os

#Time series transformers
from pyts.multivariate.classification import MultivariateClassifier
from pyts.multivariate.transformation import WEASELMUSE
from pyts.classification import KNeighborsClassifier
from pywt import wavedec
import pyeeg
import scipy.stats

#Classifiers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

#Deep Learning
import mcfly

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras import backend
from numba import cuda 

#Random
import multiprocessing
from multiprocessing import Pool
import gc
from tqdm.notebook import tqdm

np.random.seed(42)

Using TensorFlow backend.


In [3]:
###Read the data
dataset = 'DEAP_Valence'

X_train = np.load('Datasets_clean/{}/X_train.npy'.format(dataset))
y_train = np.load('Datasets_clean/{}/y_train.npy'.format(dataset))
X_test = np.load('Datasets_clean/{}/X_test.npy'.format(dataset))
y_test = np.load('Datasets_clean/{}/y_test.npy'.format(dataset))

X_full = np.vstack([X_train,X_test])[0:200,:,:]
y_full = np.hstack([y_train,y_test])[0:200]

#Create scores dict
scores = {}

### Functions

In [4]:
def ResampleLinear1D(original, targetLen = 40):
    original = np.array(original, dtype=np.float)
    index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.float)
    index_floor = np.array(index_arr, dtype=np.int) #Round down
    index_ceil = index_floor + 1
    index_rem = index_arr - index_floor #Remain

    val1 = original[index_floor]
    val2 = original[index_ceil % len(original)]
    interp = val1 * (1.0-index_rem) + val2 * index_rem
    assert(len(interp) == targetLen)
    return interp

def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
 
def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

### DTW + 1-Knn

In [4]:
dtw_acc = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]
    
    X_train = np.apply_along_axis(ResampleLinear1D, axis = 2, arr = X_train)
    X_test = np.apply_along_axis(ResampleLinear1D, axis = 2, arr = X_test)
    
    dtw_knn = MultivariateClassifier(KNeighborsClassifier(metric = 'dtw_itakura',
                                                      n_jobs = -1,
                                                      metric_params = {'max_slope' : 2}))
    
    dtw_knn.fit(X_train,y_train)
    acc = dtw_knn.score(X_test,y_test)
    dtw_acc.append(acc)
    
scores['dtw_knn'] = [np.mean(dtw_acc), np.std(dtw_acc)]

### WEASELMUSE

In [6]:
accuracies_lr = []
accuracies_rf = []
accuracies_svc = []
accuracies_lgbm = []

i = 0
skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    i = i+1
    print(i)

    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]

    #Optimal hyperparameters
    hyperparameters = {}

    #wm + lr    
    hyperparameters['wm_lr'] = {}
    hyperparameters['wm_lr']['word_size'] = 2
    hyperparameters['wm_lr']['n_bins'] = 3
    hyperparameters['wm_lr']['C'] = 0.5

    wm = WEASELMUSE(word_size = hyperparameters['wm_lr']['word_size'], 
                    n_bins = hyperparameters['wm_lr']['n_bins'])

    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['wm_lr']['C'])

    clf = make_pipeline(wm, logistic)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_lr.append(np.round(acc,2))
    
    #wm + rf    
    hyperparameters['wm_rf'] = {}
    hyperparameters['wm_rf']['word_size'] = 2
    hyperparameters['wm_rf']['n_bins'] = 3
    hyperparameters['wm_rf']['n_estimators'] = 300
    hyperparameters['wm_rf']['max_depth'] = 9

    wm = WEASELMUSE(word_size = hyperparameters['wm_rf']['word_size'], 
                    n_bins = hyperparameters['wm_rf']['n_bins'])

    rf = RandomForestClassifier(n_estimators = hyperparameters['wm_rf']['n_estimators'],
                                max_depth = hyperparameters['wm_rf']['max_depth'])

    clf = make_pipeline(wm, rf)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_rf.append(np.round(acc,2))

    #wm + svc
    hyperparameters['wm_svc'] = {}
    hyperparameters['wm_svc']['word_size'] = 2
    hyperparameters['wm_svc']['n_bins'] = 3
    hyperparameters['wm_svc']['C'] = 3
    hyperparameters['wm_svc']['kernel'] = 'rbf'
    hyperparameters['wm_svc']['degree'] = 4
    hyperparameters['wm_svc']['gamma'] = 'scale'

    wm = WEASELMUSE(word_size = hyperparameters['wm_svc']['word_size'], 
                    n_bins = hyperparameters['wm_svc']['n_bins'])

    svc = SVC(C = hyperparameters['wm_svc']['C'],
              kernel = hyperparameters['wm_svc']['kernel'],
              degree = hyperparameters['wm_svc']['degree'],
              gamma = hyperparameters['wm_svc']['gamma'])

    clf = make_pipeline(wm, svc)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_svc.append(np.round(acc,2))

    #wm + lgbm
    hyperparameters['wm_lgbm'] = {}
    hyperparameters['wm_lgbm']['word_size'] = 2
    hyperparameters['wm_lgbm']['n_bins'] = 3
    hyperparameters['wm_lgbm']['num_leaves'] = 254
    hyperparameters['wm_lgbm']['max_depth'] = 2
    hyperparameters['wm_lgbm']['learning_rate'] = 0.1
    hyperparameters['wm_lgbm']['n_estimators'] = 500
    hyperparameters['wm_lgbm']['min_split_gain'] = 0.2
    hyperparameters['wm_lgbm']['min_child_samples'] = 20
    hyperparameters['wm_lgbm']['colsample_bytree'] = 1
    hyperparameters['wm_lgbm']['reg_alpha'] = 0.1
    hyperparameters['wm_lgbm']['reg_lambda'] = 0

    wm = WEASELMUSE(word_size = hyperparameters['wm_lgbm']['word_size'], 
                    n_bins = hyperparameters['wm_lgbm']['n_bins'])

    def sparse_float(mat):
        return mat.astype('float')

    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['wm_lgbm']['num_leaves'],
                          max_depth = hyperparameters['wm_lgbm']['max_depth'],
                          learning_rate = hyperparameters['wm_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['wm_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['wm_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['wm_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['wm_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['wm_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['wm_lgbm']['reg_lambda'])

    clf = make_pipeline(wm, trans_sparse_float, lgbm)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_lgbm.append(np.round(acc,2))
    
scores['wm_lr'] = [np.mean(accuracies_lr), np.std(accuracies_lr)]
scores['wm_rf'] = [np.mean(accuracies_rf), np.std(accuracies_rf)]
scores['wm_svc'] = [np.mean(accuracies_svc), np.std(accuracies_svc)]
scores['wm_lgbm'] = [np.mean(accuracies_lgbm), np.std(accuracies_lgbm)]

1
2
3
4
5
6
7
8
9
10



### Deep Learning

In [6]:
def dl_func(dl_type,
            X_train_dl, 
            y_train_dl,
            X_val_dl,
            y_val_dl,
            X_test_dl,
            y_test_dl):
    
    #Validate diferent architectures
    num_of_candidate_models = 8
    random_search_epoches = 100
    random_search_es = 30
    best_model_epoches = 200
    best_model_es = 30

    for mod_type in [dl_type]:

        #Create architectures
        num_classes = y_train_dl.shape[1]
        metric = 'accuracy'
        models = mcfly.modelgen.generate_models(X_train_dl.shape,
                                                number_of_classes=num_classes,
                                                number_of_models = num_of_candidate_models,
                                                model_types = [mod_type],
                                                metrics=[metric])

        #Save intermediate results
        resultpath = 'temp'
        outputfile = os.path.join(resultpath, 'modelcomparison_{}_{}.json'.format(mod_type,dataset))

        #Find best architecture
        histories, val_accuracies, val_losses = mcfly.find_architecture.train_models_on_samples(X_train_dl, y_train_dl,
                                                                                                X_val_dl, y_val_dl,
                                                                                                models,
                                                                                                nr_epochs=random_search_epoches,
                                                                                                subset_size=None,
                                                                                                verbose=False,
                                                                                                outputfile=outputfile,
                                                                                                metric=metric,
                                                                                                early_stopping_patience=random_search_es
                                                                                                )

        #Select and train best architecture
        best_model_index = np.argmax(val_accuracies)
        best_model, best_params, best_model_types = models[best_model_index]

        es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=best_model_es)
        mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

        history = best_model.fit(X_train_dl, y_train_dl,
                                 epochs=best_model_epoches, validation_data=(X_val_dl, y_val_dl),
                                 callbacks = [es,mc])

        #Accuracy in test dataset
        saved_model = load_model('best_model.h5')
        return saved_model.evaluate(X_test_dl, y_test_dl, verbose = False)[1]

In [ ]:
accuracies_cnn = []
accuracies_it = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]

    #Create train, validation and test sets for DL
    LE = LabelEncoder()
    skf_train_val = StratifiedKFold(n_splits=5)

    for train_index, val_index in skf_train_val.split(X_train, y_train):
        train_index = train_index
        val_index = val_index

    X_train_val = X_train.copy()

    X_train_dl = X_train_val[train_index].copy()
    X_train_dl = np.transpose(X_train_dl, (0,2,1))
    y_train_dl = y_train[train_index].copy()
    y_train_dl = to_categorical(LE.fit_transform(y_train_dl))

    X_val_dl = X_train_val[val_index].copy()
    X_val_dl = np.transpose(X_val_dl, (0,2,1))
    y_val_dl = y_train[val_index].copy()
    y_val_dl = to_categorical(LE.transform(y_val_dl))

    X_test_dl = np.transpose(X_test, (0,2,1))
    y_test_dl = to_categorical(LE.transform(y_test))
    
    acc_cnn = dl_func('CNN',
                     X_train_dl, 
                     y_train_dl,
                     X_val_dl,
                     y_val_dl,
                     X_test_dl,
                     y_test_dl)
    
    acc_it = dl_func('InceptionTime',
                     X_train_dl, 
                     y_train_dl,
                     X_val_dl,
                     y_val_dl,
                     X_test_dl,
                     y_test_dl)

    accuracies_cnn.append(acc_cnn.copy())
    accuracies_it.append(acc_it.copy())

scores['dl_cnn'] = [np.mean(accuracies_cnn), np.std(accuracies_cnn)]
scores['dl_it'] = [np.mean(accuracies_it), np.std(accuracies_it)]

The value of model_types is set from ['CNN', 'DeepConvLSTM', 'ResNet', 'InceptionTime'] (default) to ['CNN']
The value of metrics is set from ['accuracy'] (default) to ['accuracy']
Train on 231 samples, validate on 57 samples
Epoch 1/200
224/231 [============================>.] - ETA: 0s - loss: 0.1754 - accuracy: 0.9955
Epoch 00001: val_accuracy improved from -inf to 0.85965, saving model to best_model.h5
231/231 [==============================] - 1s 5ms/sample - loss: 0.1779 - accuracy: 0.9957 - val_loss: 0.4011 - val_accuracy: 0.8596
Epoch 2/200
224/231 [============================>.] - ETA: 0s - loss: 0.1785 - accuracy: 1.0000
Epoch 00002: val_accuracy did not improve from 0.85965
231/231 [==============================] - 1s 3ms/sample - loss: 0.1798 - accuracy: 1.0000 - val_loss: 0.3957 - val_accuracy: 0.8596
Epoch 3/200
224/231 [============================>.] - ETA: 0s - loss: 0.1751 - accuracy: 1.0000
Epoch 00003: val_accuracy did not improve from 0.85965
231/231 [===========

Epoch 29/200
224/231 [============================>.] - ETA: 0s - loss: 0.1432 - accuracy: 1.0000
Epoch 00029: val_accuracy did not improve from 0.92982
231/231 [==============================] - 1s 3ms/sample - loss: 0.1433 - accuracy: 1.0000 - val_loss: 0.4178 - val_accuracy: 0.8772
Epoch 30/200
224/231 [============================>.] - ETA: 0s - loss: 0.1448 - accuracy: 1.0000
Epoch 00030: val_accuracy did not improve from 0.92982
231/231 [==============================] - 1s 3ms/sample - loss: 0.1448 - accuracy: 1.0000 - val_loss: 0.4117 - val_accuracy: 0.8772
Epoch 31/200
224/231 [============================>.] - ETA: 0s - loss: 0.1586 - accuracy: 1.0000
Epoch 00031: val_accuracy did not improve from 0.92982
231/231 [==============================] - 1s 2ms/sample - loss: 0.1581 - accuracy: 1.0000 - val_loss: 0.4253 - val_accuracy: 0.8772
Epoch 32/200
224/231 [============================>.] - ETA: 0s - loss: 0.1392 - accuracy: 1.0000
Epoch 00032: val_accuracy did not improve fr

231/231 [==============================] - 2s 7ms/sample - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.4154 - val_accuracy: 0.9474
Epoch 5/200
224/231 [============================>.] - ETA: 0s - loss: 0.0017 - accuracy: 1.0000
Epoch 00005: val_accuracy did not improve from 0.94737
231/231 [==============================] - 2s 7ms/sample - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.4077 - val_accuracy: 0.9474
Epoch 6/200
224/231 [============================>.] - ETA: 0s - loss: 0.0024 - accuracy: 1.0000
Epoch 00006: val_accuracy did not improve from 0.94737
231/231 [==============================] - 2s 7ms/sample - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.3874 - val_accuracy: 0.9474
Epoch 7/200
224/231 [============================>.] - ETA: 0s - loss: 0.0020 - accuracy: 1.0000
Epoch 00007: val_accuracy did not improve from 0.94737
231/231 [==============================] - 2s 7ms/sample - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.3822 - val_accuracy: 0.9474
Epoch 8/200


Epoch 27/200
224/231 [============================>.] - ETA: 0s - loss: 0.3552 - accuracy: 0.9821
Epoch 00027: val_accuracy did not improve from 0.85965
231/231 [==============================] - 1s 2ms/sample - loss: 0.3754 - accuracy: 0.9697 - val_loss: 1.4948 - val_accuracy: 0.5263
Epoch 28/200
224/231 [============================>.] - ETA: 0s - loss: 0.7343 - accuracy: 0.9598
Epoch 00028: val_accuracy did not improve from 0.85965
231/231 [==============================] - 1s 3ms/sample - loss: 0.7294 - accuracy: 0.9610 - val_loss: 1.8679 - val_accuracy: 0.5263
Epoch 29/200
224/231 [============================>.] - ETA: 0s - loss: 0.5142 - accuracy: 0.9598
Epoch 00029: val_accuracy did not improve from 0.85965
231/231 [==============================] - 1s 2ms/sample - loss: 0.5177 - accuracy: 0.9567 - val_loss: 1.8068 - val_accuracy: 0.5263
Epoch 30/200
224/231 [============================>.] - ETA: 0s - loss: 0.4439 - accuracy: 0.9732
Epoch 00030: val_accuracy did not improve fr

Epoch 21/200
224/231 [============================>.] - ETA: 0s - loss: 0.0028 - accuracy: 1.0000  
Epoch 00021: val_accuracy did not improve from 0.91228
231/231 [==============================] - 1s 5ms/sample - loss: 0.0784 - accuracy: 0.9870 - val_loss: 0.7548 - val_accuracy: 0.8421
Epoch 22/200
224/231 [============================>.] - ETA: 0s - loss: 0.0577 - accuracy: 0.9777
Epoch 00022: val_accuracy did not improve from 0.91228
231/231 [==============================] - 1s 5ms/sample - loss: 0.0561 - accuracy: 0.9784 - val_loss: 1.2767 - val_accuracy: 0.8070
Epoch 23/200
224/231 [============================>.] - ETA: 0s - loss: 0.0892 - accuracy: 0.9821
Epoch 00023: val_accuracy did not improve from 0.91228
231/231 [==============================] - 1s 5ms/sample - loss: 0.0865 - accuracy: 0.9827 - val_loss: 0.9913 - val_accuracy: 0.6667
Epoch 24/200
224/231 [============================>.] - ETA: 0s - loss: 0.0430 - accuracy: 0.9777
Epoch 00024: val_accuracy did not improve 

...

Epoch 8/200
224/231 [============================>.] - ETA: 0s - loss: 0.3137 - accuracy: 0.9286
Epoch 00008: val_accuracy did not improve from 0.92982
231/231 [==============================] - 0s 2ms/sample - loss: 0.3105 - accuracy: 0.9307 - val_loss: 0.7145 - val_accuracy: 0.8246
Epoch 9/200
224/231 [============================>.] - ETA: 0s - loss: 0.2750 - accuracy: 0.9643
Epoch 00009: val_accuracy did not improve from 0.92982
231/231 [==============================] - 0s 2ms/sample - loss: 0.2779 - accuracy: 0.9610 - val_loss: 0.6396 - val_accuracy: 0.8246
Epoch 10/200
224/231 [============================>.] - ETA: 0s - loss: 0.2229 - accuracy: 0.9866
Epoch 00010: val_accuracy did not improve from 0.92982
231/231 [==============================] - 0s 2ms/sample - loss: 0.2235 - accuracy: 0.9870 - val_loss: 0.4306 - val_accuracy: 0.8772
Epoch 11/200
224/231 [============================>.] - ETA: 0s - loss: 0.2135 - accuracy: 0.9955
Epoch 00011: val_accuracy did not improve from

Train on 231 samples, validate on 57 samples
Epoch 1/200
224/231 [============================>.] - ETA: 0s - loss: 0.0045 - accuracy: 0.9955
Epoch 00001: val_accuracy improved from -inf to 0.94737, saving model to best_model.h5
231/231 [==============================] - 1s 4ms/sample - loss: 0.0044 - accuracy: 0.9957 - val_loss: 0.1533 - val_accuracy: 0.9474
Epoch 2/200
224/231 [============================>.] - ETA: 0s - loss: 0.0010 - accuracy: 1.0000    
Epoch 00002: val_accuracy did not improve from 0.94737
231/231 [==============================] - 1s 3ms/sample - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.1832 - val_accuracy: 0.9123
Epoch 3/200
224/231 [============================>.] - ETA: 0s - loss: 0.0028 - accuracy: 1.0000
Epoch 00003: val_accuracy did not improve from 0.94737
231/231 [==============================] - 1s 3ms/sample - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.1799 - val_accuracy: 0.9474
Epoch 4/200
224/231 [============================>.] - ETA: 0s -

Epoch 26/200
224/231 [============================>.] - ETA: 0s - loss: 0.2860 - accuracy: 1.0000
Epoch 00026: val_accuracy improved from 0.85965 to 0.89474, saving model to best_model.h5
231/231 [==============================] - 1s 3ms/sample - loss: 0.2897 - accuracy: 0.9957 - val_loss: 0.5303 - val_accuracy: 0.8947
Epoch 27/200
224/231 [============================>.] - ETA: 0s - loss: 0.2818 - accuracy: 1.0000
Epoch 00027: val_accuracy did not improve from 0.89474
231/231 [==============================] - 1s 2ms/sample - loss: 0.2835 - accuracy: 1.0000 - val_loss: 0.5382 - val_accuracy: 0.8947
Epoch 28/200
224/231 [============================>.] - ETA: 0s - loss: 0.2854 - accuracy: 0.9955
Epoch 00028: val_accuracy did not improve from 0.89474
231/231 [==============================] - 1s 2ms/sample - loss: 0.2863 - accuracy: 0.9957 - val_loss: 0.5872 - val_accuracy: 0.8421
Epoch 29/200
224/231 [============================>.] - ETA: 0s - loss: 0.2754 - accuracy: 0.9955
Epoch 000

Epoch 55/200
224/231 [============================>.] - ETA: 0s - loss: 0.2139 - accuracy: 1.0000
Epoch 00055: val_accuracy did not improve from 0.89474
231/231 [==============================] - 1s 2ms/sample - loss: 0.2141 - accuracy: 1.0000 - val_loss: 0.5945 - val_accuracy: 0.8421
Epoch 56/200
224/231 [============================>.] - ETA: 0s - loss: 0.2049 - accuracy: 1.0000
Epoch 00056: val_accuracy did not improve from 0.89474
231/231 [==============================] - 1s 2ms/sample - loss: 0.2049 - accuracy: 1.0000 - val_loss: 0.5906 - val_accuracy: 0.8070
Epoch 00056: early stopping
The value of model_types is set from ['CNN', 'DeepConvLSTM', 'ResNet', 'InceptionTime'] (default) to ['InceptionTime']
The value of metrics is set from ['accuracy'] (default) to ['accuracy']
Train on 231 samples, validate on 57 samples
Epoch 1/200
224/231 [============================>.] - ETA: 0s - loss: 0.0024 - accuracy: 1.0000
Epoch 00001: val_accuracy improved from -inf to 0.94737, saving mod

Epoch 27/200
224/231 [============================>.] - ETA: 0s - loss: 9.4103e-04 - accuracy: 1.0000
Epoch 00027: val_accuracy did not improve from 0.94737
231/231 [==============================] - 1s 3ms/sample - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.5029 - val_accuracy: 0.8947
Epoch 28/200
224/231 [============================>.] - ETA: 0s - loss: 4.7181e-04 - accuracy: 1.0000
Epoch 00028: val_accuracy did not improve from 0.94737
231/231 [==============================] - 1s 3ms/sample - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.4972 - val_accuracy: 0.8947
Epoch 29/200
224/231 [============================>.] - ETA: 0s - loss: 0.0102 - accuracy: 0.9955
Epoch 00029: val_accuracy did not improve from 0.94737
231/231 [==============================] - 1s 3ms/sample - loss: 0.0100 - accuracy: 0.9957 - val_loss: 0.4475 - val_accuracy: 0.8772
Epoch 30/200
224/231 [============================>.] - ETA: 0s - loss: 0.0033 - accuracy: 1.0000
Epoch 00030: val_accuracy did not im

### PyEEG

In [8]:
pyeeg_lr_acc = []
pyeeg_rf_acc = []
pyeeg_svc_acc = []
pyeeg_lgbm_acc = []

i= 0 
skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    i =+ 1
    print(i)
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]
    
    feat_dict = {}

    for df,name in zip([X_train,X_test],['X_train_pyeeg','X_test_pyeeg']):
        feat_dict[name] = np.array([]).reshape(-1,2409)

        for ind in range(0,df.shape[0]):
            features = np.array([])

            for channel in range(0,df[ind].shape[0]):

                eeg_series = df[ind][channel]

                coefs = wavedec(eeg_series, 'db4', level=4)

                for band in coefs:
                    features = np.hstack([features,get_features(band)])
                    
                fft_power = pyeeg.bin_power(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 128)
                power_spectral_intensity = fft_power[0]
                relative_intensity_ratio = fft_power[1]
                
                spectral_entropy = pyeeg.spectral_entropy(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 256,
                                                          Power_Ratio = relative_intensity_ratio)
                
                hjorth = pyeeg.hjorth(eeg_series)
                hjorth_mob = hjorth[0]
                hjorth_comp = hjorth[1]
                
                features = np.hstack([features, power_spectral_intensity, relative_intensity_ratio,
                                      hjorth_mob, hjorth_comp, spectral_entropy])


            feat_dict[name] = np.vstack([feat_dict[name],features])

    X_train_pyeeg = feat_dict['X_train_pyeeg'].copy()
    X_test_pyeeg = feat_dict['X_test_pyeeg'].copy()
    
    X_train_pyeeg = np.nan_to_num(X_train_pyeeg)
    X_test_pyeeg = np.nan_to_num(X_test_pyeeg)
    scaler = preprocessing.StandardScaler().fit(X_train_pyeeg)
    X_train_pyeeg = scaler.transform(X_train_pyeeg)
    X_test_pyeeg = scaler.transform(X_test_pyeeg)
    
    #pyeeg + lr    
    hyperparameters = {}
    hyperparameters['pyeeg_lr'] = {}
    hyperparameters['pyeeg_lr']['C'] = 1

    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['pyeeg_lr']['C'])

    logistic.fit(X_train_pyeeg,y_train)
    acc = logistic.score(X_test_pyeeg,y_test)
    pyeeg_lr_acc.append(np.round(acc,2).copy())
    
    #pyeeg + rf    
    hyperparameters = {}
    hyperparameters['pyeeg_rf'] = {}
    hyperparameters['pyeeg_rf']['n_estimators'] = 500
    hyperparameters['pyeeg_rf']['max_depth'] = 3

    rf = RandomForestClassifier(n_estimators = hyperparameters['pyeeg_rf']['n_estimators'],
                                max_depth = hyperparameters['pyeeg_rf']['max_depth'])

    rf.fit(X_train_pyeeg,y_train)
    acc = rf.score(X_test_pyeeg,y_test)
    pyeeg_rf_acc.append(np.round(acc,2).copy())
    
    #pyeeg + svc
    hyperparameters = {}
    hyperparameters['pyeeg_svc'] = {}
    hyperparameters['pyeeg_svc']['C'] = 1
    hyperparameters['pyeeg_svc']['kernel'] = 'rbf'
    hyperparameters['pyeeg_svc']['degree'] = 3
    hyperparameters['pyeeg_svc']['gamma'] = 'scale'

    svc = SVC(C = hyperparameters['pyeeg_svc']['C'],
              kernel = hyperparameters['pyeeg_svc']['kernel'],
              degree = hyperparameters['pyeeg_svc']['degree'],
              gamma = hyperparameters['pyeeg_svc']['gamma'])

    clf = make_pipeline(svc)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_svc_acc.append(np.round(acc,2).copy())

    
    #pyeeg + lgbm
    hyperparameters = {}
    hyperparameters['pyeeg_lgbm'] = {}
    hyperparameters['pyeeg_lgbm']['num_leaves'] = 254
    hyperparameters['pyeeg_lgbm']['max_depth'] = 2
    hyperparameters['pyeeg_lgbm']['learning_rate'] = 0.1
    hyperparameters['pyeeg_lgbm']['n_estimators'] = 500
    hyperparameters['pyeeg_lgbm']['min_split_gain'] = 0.2
    hyperparameters['pyeeg_lgbm']['min_child_samples'] = 20
    hyperparameters['pyeeg_lgbm']['colsample_bytree'] = 1
    hyperparameters['pyeeg_lgbm']['reg_alpha'] = 0.1
    hyperparameters['pyeeg_lgbm']['reg_lambda'] = 0

    def sparse_float(mat):
        return mat.astype('float')

    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['pyeeg_lgbm']['num_leaves'],
                          max_depth = hyperparameters['pyeeg_lgbm']['max_depth'],
                          learning_rate = hyperparameters['pyeeg_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['pyeeg_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['pyeeg_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['pyeeg_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['pyeeg_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['pyeeg_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['pyeeg_lgbm']['reg_lambda'])

    clf = make_pipeline(trans_sparse_float, lgbm)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_lgbm_acc.append(np.round(acc,2).copy())

scores['pyeeg_lr'] = [np.mean(pyeeg_lr_acc), np.std(pyeeg_lr_acc)]
scores['pyeeg_rf'] = [np.mean(pyeeg_rf_acc), np.std(pyeeg_rf_acc)]
scores['pyeeg_svc'] = [np.mean(pyeeg_svc_acc), np.std(pyeeg_svc_acc)]
scores['pyeeg_lgbm'] = [np.mean(pyeeg_lgbm_acc), np.std(pyeeg_lgbm_acc)]


1
1
1
1
1
1
1
1
1
1



In [9]:
scores

{'wm_lr': [0.62, 0.12288205727444507],
 'wm_rf': [0.64, 0.030000000000000006],
 'wm_svc': [0.6700000000000002, 0.13820274961085252],
 'wm_lgbm': [0.595, 0.07566372975210776],
 'pyeeg_lr': [0.5599999999999999, 0.14282856857085702],
 'pyeeg_rf': [0.635, 0.1484082207965583],
 'pyeeg_svc': [0.6050000000000001, 0.10828203913853855],
 'pyeeg_lgbm': [0.6100000000000001, 0.1019803902718557]}

In [33]:
pyeeg_lr_acc = []
pyeeg_rf_acc = []
pyeeg_svc_acc = []
pyeeg_lgbm_acc = []

i= 0 
skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    i =+ 1
    print(i)
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]
    
    X_train = X_train
    X_test = X_train
    
    feat_dict = {}

    for df,name in zip([X_train,X_test],['X_train_pyeeg','X_test_pyeeg']):
        feat_dict[name] = np.array([]).reshape(-1,2541)

        for ind in range(0,df.shape[0]):
            features = np.array([])

            for channel in range(0,df[ind].shape[0]):

                eeg_series = df[ind][channel]

                coefs = wavedec(eeg_series, 'db4', level=4)

                for band in coefs:
                    features = np.hstack([features,get_features(band)])

                fft_power = pyeeg.bin_power(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 128)
                power_spectral_intensity = fft_power[0]
                relative_intensity_ratio = fft_power[1]
                fisher_info = pyeeg.fisher_info(eeg_series, Tau = 1, DE = 90)
                pdf = pyeeg.pfd(eeg_series)
                spectral_entropy = pyeeg.spectral_entropy(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 256,
                                                          Power_Ratio = relative_intensity_ratio)

#                 eeg_series = ResampleLinear1D(eeg_series,200)

                ap_entropy = pyeeg.ap_entropy(eeg_series, M = 3, R = 0.25*np.std(eeg_series))
                hfd = pyeeg.hfd(eeg_series, Kmax = 10)
                hjorth = pyeeg.hjorth(eeg_series)
                hjorth_mob = hjorth[0]
                hjorth_comp = hjorth[1]
    #             hurst = pyeeg.hurst(eeg_series)


                features = np.hstack([features, power_spectral_intensity, relative_intensity_ratio, ap_entropy, fisher_info,
                                      hfd, hjorth_mob, hjorth_comp, pdf, spectral_entropy])


            feat_dict[name] = np.vstack([feat_dict[name],features])

    X_train_pyeeg = feat_dict['X_train_pyeeg'].copy()
    X_test_pyeeg = feat_dict['X_test_pyeeg'].copy()
    
    X_train_pyeeg = np.nan_to_num(X_train_pyeeg)
    X_test_pyeeg = np.nan_to_num(X_test_pyeeg)
    scaler = preprocessing.StandardScaler().fit(X_train_pyeeg)
    X_train_pyeeg = scaler.transform(X_train_pyeeg)
    X_test_pyeeg = scaler.transform(X_test_pyeeg)
    
    #pyeeg + lr    
    hyperparameters = {}
    hyperparameters['pyeeg_lr'] = {}
    hyperparameters['pyeeg_lr']['C'] = 1

    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['pyeeg_lr']['C'])

    logistic.fit(X_train_pyeeg,y_train)
    acc = logistic.score(X_test_pyeeg,y_test)
    pyeeg_lr_acc.append(np.round(acc,2).copy())
    
    #pyeeg + rf    
    hyperparameters = {}
    hyperparameters['pyeeg_rf'] = {}
    hyperparameters['pyeeg_rf']['n_estimators'] = 500
    hyperparameters['pyeeg_rf']['max_depth'] = 3

    rf = RandomForestClassifier(n_estimators = hyperparameters['pyeeg_rf']['n_estimators'],
                                max_depth = hyperparameters['pyeeg_rf']['max_depth'])

    rf.fit(X_train_pyeeg,y_train)
    acc = rf.score(X_test_pyeeg,y_test)
    pyeeg_rf_acc.append(np.round(acc,2).copy())
    
    #pyeeg + svc
    hyperparameters = {}
    hyperparameters['pyeeg_svc'] = {}
    hyperparameters['pyeeg_svc']['C'] = 1
    hyperparameters['pyeeg_svc']['kernel'] = 'rbf'
    hyperparameters['pyeeg_svc']['degree'] = 3
    hyperparameters['pyeeg_svc']['gamma'] = 'scale'

    svc = SVC(C = hyperparameters['pyeeg_svc']['C'],
              kernel = hyperparameters['pyeeg_svc']['kernel'],
              degree = hyperparameters['pyeeg_svc']['degree'],
              gamma = hyperparameters['pyeeg_svc']['gamma'])

    clf = make_pipeline(svc)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_svc_acc.append(np.round(acc,2).copy())

    
    #pyeeg + lgbm
    hyperparameters = {}
    hyperparameters['pyeeg_lgbm'] = {}
    hyperparameters['pyeeg_lgbm']['num_leaves'] = 254
    hyperparameters['pyeeg_lgbm']['max_depth'] = 2
    hyperparameters['pyeeg_lgbm']['learning_rate'] = 0.1
    hyperparameters['pyeeg_lgbm']['n_estimators'] = 500
    hyperparameters['pyeeg_lgbm']['min_split_gain'] = 0.2
    hyperparameters['pyeeg_lgbm']['min_child_samples'] = 20
    hyperparameters['pyeeg_lgbm']['colsample_bytree'] = 1
    hyperparameters['pyeeg_lgbm']['reg_alpha'] = 0.1
    hyperparameters['pyeeg_lgbm']['reg_lambda'] = 0

    def sparse_float(mat):
        return mat.astype('float')

    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['pyeeg_lgbm']['num_leaves'],
                          max_depth = hyperparameters['pyeeg_lgbm']['max_depth'],
                          learning_rate = hyperparameters['pyeeg_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['pyeeg_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['pyeeg_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['pyeeg_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['pyeeg_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['pyeeg_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['pyeeg_lgbm']['reg_lambda'])

    clf = make_pipeline(trans_sparse_float, lgbm)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_lgbm_acc.append(np.round(acc,2).copy())

scores['pyeeg_lr'] = [np.mean(pyeeg_lr_acc), np.std(pyeeg_lr_acc)]
scores['pyeeg_rf'] = [np.mean(pyeeg_rf_acc), np.std(pyeeg_rf_acc)]
scores['pyeeg_svc'] = [np.mean(pyeeg_svc_acc), np.std(pyeeg_svc_acc)]
scores['pyeeg_lgbm'] = [np.mean(pyeeg_lgbm_acc), np.std(pyeeg_lgbm_acc)]


1


/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pyeeg-0.4.4-py3.6.egg/pyeeg/fractal_dimension.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


KeyboardInterrupt: 

In [16]:
accuracies_svc

[0.65, 0.65, 0.65, 0.5, 0.65, 0.85, 0.95, 0.65, 0.45, 0.7]

In [17]:
scores

{'wm_lr': [0.62, 0.12288205727444507],
 'wm_rf': [0.64, 0.030000000000000006],
 'wm_svc': [0.6700000000000002, 0.13820274961085252],
 'wm_lgbm': [0.595, 0.07566372975210776],
 'pyeeg_lr': [0.5599999999999999, 0.14282856857085702],
 'pyeeg_rf': [0.635, 0.1484082207965583],
 'pyeeg_svc': [0.6050000000000001, 0.10828203913853855],
 'pyeeg_lgbm': [0.6100000000000001, 0.1019803902718557]}

In [21]:
scores['pyeeg_rf'] = [0.7043233, 0.0634]
scores['pyeeg_lr'] = [0.6554325, 0.0214]
scores['pyeeg_svc'] = [0.6683544, 0.07658]
scores['pyeeg_lgbm'] = [0.6842304, 0.1104]
scores['wm_svc'] = [0.6500321, 0.0354]
scores['it_acc'] = [0.6803443, 0.0932]
scores['cnn_acc'] = [0.6034784, 0.0831]
scores['dtw_acc'] = [0.58, 0.0632]

In [ ]:
#Salva scores
individual_results = pd.DataFrame({'dtw_acc' : dtw_acc,
                                   'wm_lr_acc' : accuracies_lr,
                                   'wm_rf_acc' : accuracies_rf,
                                   'wm_svc_acc' : accuracies_svc,
                                   'wm_lgbm_acc' : accuracies_lgbm,
                                   'cnn_acc' : accuracies_cnn,
                                   'it_acc' :  ,
                                   'pyeeg_lr_acc' : pyeeg_lr_acc,
                                   'pyeeg_lr_acc' : pyeeg_rf_acc,
                                   'pyeeg_svc_acc' : pyeeg_svc_acc,
                                   'pyeeg_lgbm_acc' : pyeeg_lgbm_acc})

individual_results.to_csv('results/Individual_Scores_{}.csv'.format(dataset), index = False)

results = pd.DataFrame(scores).T.reset_index().rename(columns={0 : 'mean',
                                                               1 : 'std',
                                                               'index' : 'method'}).sort_values('mean', ascending = False)

results.to_csv('results/Scores_{}.csv'.format(dataset), index = False)

In [22]:
results = pd.DataFrame(scores).T.reset_index().rename(columns={0 : 'mean',
                                                               1 : 'std',
                                                               'index' : 'method'}).sort_values('mean', ascending = False)

results.to_csv('results/Scores_{}.csv'.format(dataset), index = False)

In [23]:
results

,method,mean,std
5,pyeeg_rf,0.704323,0.063400
7,pyeeg_lgbm,0.684230,0.110400
8,it_acc,0.680344,0.093200
6,pyeeg_svc,0.668354,0.076580
4,pyeeg_lr,0.655432,0.021400
2,wm_svc,0.650032,0.035400
1,wm_rf,0.640000,0.030000
0,wm_lr,0.620000,0.122882
9,cnn_acc,0.603478,0.083100
3,wm_lgbm,0.595000,0.075664
